# Análisis de Conglomerados (Cluster Analysis) - Caso Práctico

**Fuente:** [Universidad de Murcia](https://www.um.es/estadempresa/multivarianteR/CLUS/caso_CLUS.html)

## Introducción

El análisis de conglomerados se utiliza para organizar un conjunto de observaciones en grupos, de manera que la asociación sea alta entre miembros del mismo grupo y baja entre miembros de grupos diferentes. En este caso, agruparemos las Comunidades Autónomas de España según variables socioeconómicas del año 2014.

### Variables del problema:

* `tasainm`: Tasa bruta de inmigración.
* `tasaem`: Tasa bruta de emigración.
* `tasaact`: Tasa de actividad.
* `tasaempleo`: Tasa de empleo.
* `tasaparo`: Tasa de paro.
* `ipc`: IPC base 2016.
* `pibpc`: PIB a precios corrientes por individuo.


## Etapa 1. Exploración de los datos

Primero, cargamos los datos y realizamos un resumen estadístico.

In [ ]:
# Cargar datos (asegúrate de tener el archivo conglomerados.RData en tu directorio)
load("data/conglomerados.RData")

# 2. Reparar la codificación de los nombres de las filas
# Esto convierte cualquier residuo de Latin-1 a UTF-8 real
rownames(ccaa) <- iconv(rownames(ccaa), from = "latin1", to = "UTF-8")

# 3. Tipificar (esto heredará los nombres ya limpios)
ccaa_tip <- scale(ccaa)

# 4. Verificar (si ves caracteres extraños aquí, sabremos que el origen es la codificación)
print(rownames(ccaa_tip))




In [ ]:
# Resumen de los datos
summary(ccaa)

Dado que las variables tienen escalas muy distintas (ej. `pibpc` frente a `tasainm`), es necesario tipificar los datos para que todas las variables tengan el mismo peso.

In [ ]:
ccaa_tip <- scale(ccaa)

---

## Etapa 2. Detección de valores atípicos

Utilizamos diagramas de caja para identificar valores extremos.

In [ ]:
par(cex.axis=0.8) 
bx1 <- boxplot(ccaa_tip, col = "lightyellow")

# OPCIONAL: utlizar lo siguiente solo si quiere colocar etiquetas a los valores atípicos o extremos
if(length(bx1$out)!= 0){
   out.rows <- sapply(1:length(bx1$out), function(i)
   which(ccaa_tip[ ,bx1$group[i]] == bx1$out[i]))
   text(bx1$group, bx1$out, rownames(ccaa_tip)[out.rows], pos=4, cex = 0.8)
 }

*Nota: Ceuta y Melilla suelen aparecer como atípicos en tasas migratorias, y Cataluña en el IPC.*


## Etapa 3. Estudio de la correlación

Para medir el grado de relación entre cada dos variables cuantitativas, calculamos la matriz de correlaciones aplicando la función cor sobre los datos (da igual hacerlo sobre los datos originales o sobre los tipificados, ya que la correlación es independiente de la escala de medida de las variables). Recordemos que una correlación cercana a 1 o -1 indica que existe una fuerte relación lineal entre las variables, en el primer caso directa y en el segundo inversa, mientras que valores cercanos a cero indicarían ausencia de relación lineal.

In [ ]:
cor(ccaa_tip)


Observe que resulta mucho más sencillo interpretar estas matrices si no se utiliza un número de decimales tan elevado. Una manera de reducirlo es utilizando la función round, de forma que en adelante la emplearemos con dicho objetivo:



In [ ]:
round(cor(ccaa_tip), 2)


Esta matriz también se puede representar gráficamente,



In [ ]:
library(corrplot) 
corrplot.mixed(cor(ccaa_tip))

Observando la matriz de correlaciones o el gráfico anterior podemos ver que existen algunas correlaciones grandes entre las variables (por ejemplo, entre la tasa de emigración y la tasa de inmigración, o entre el PIBpc y la tasa de empleo). La cuestión es si estas correlaciones son lo suficientemente grandes para influir en la matriz de distancias cuando se utiliza la distancia euclídea.

Si consideramos que las correlaciones son suficientemente grandes para influir en la matriz de distancias sería conveniente realizar un análisis de componentes principales primero y realizar el análisis de conglomerados a partir de los resultados de este, ya que las componentes obtenidas serían incorreladas. Si pensamos que las correlaciones no son suficientemente grandes podemos realizar el análisis sobre las variables originales.

Comenzaremos suponiendo que las correlaciones no son suficientemente grandes, y lo repetiremos suponiendo que sí, para observar si las correlaciones entre las variables afectan a la formación y composición de conglomerados

---

## Etapa 4. Selección de la medida de similitud y técnica de agrupación

En primer lugar, debe elegirse la medida de distancia a emplear para decidir qué comunidades autónomas son las más parecidas y cuáles presentan mayores diferencias según las variables de interés. Entre las medidas de distancia que se pueden emplear para variables métricas, como las disponibles en este caso, se encuentran la distancia euclídea, la distancia euclídea al cuadrado, la distancia de Minkowski y la distancia city block o “Manhattan”. Cualquiera de estas distancias se puede calcular utilizando la función dist mediante el argumento method.

Si suponemos, inicialmente, que las correlaciones entre las variables de interés no son suficientemente elevadas como para realizar un análisis de componentes principales previo, procederemos utilizando la distancia euclídea aplicada sobre la matriz de datos tipificados. Esta distancia es la que se emplea con más frecuencia y está implementada por defecto en la función dist.

In [ ]:
distancia <- dist(ccaa_tip)
round(as.matrix(distancia)[1:19,1:6], 2) # visualización de parte de la matriz de distancias

Esta matriz de distancias nos muestra, por ejemplo, que Cataluña y Melilla son las regiones más disimilares (con una distancia de 6.84), mientras que Cantabria y Castilla-León son las más similares (con una distancia de 0.75).

Una vez obtenida la matriz de disimilaridades, la siguiente etapa es decidir el método de agrupación a utilizar. Vamos a comenzar planteando un análisis de conglomerados jerárquico, para luego realizar un análisis no jeráquico de k-medias.

### 1. Conglomerado Jerárquico (`hclust`)

En un método jeráquico es preciso establecer el método de agrupación de los elementos, en nuestro caso, las regiones autónomas. Existen varios métodos disponibles, como el vecino más cercano o enlace simple (single), el vecino más lejano o enlace completo (complete), agrupación de centroides o enlace centroide (centroid), enlace promedio ponderado entre grupos (average), y sin ponderar (mcquitty), y enlace de mínima varianza o método de Ward (ward.D o ward.D2). Todos estos métodos se pueden implementar dentro de la función hclust cambiando el argumento method.

A continuación, vamos a calcular los grupos utilizando tres métodos, complete (el que tiene predeterminado la función hclust), average (que viene por defecto en otros paquetes estadísticos como SPSS) y ward.D2 (que suele ser el método que mejor clasifica).




#### Método complete
Creamos un objeto de R que contenga la información que ofrece la función hclust y sobre el cual se puede dibujar:


In [ ]:
clust <- hclust(distancia, method = "complete") # Como el método por defecto es el `complete` no sería necesario especificarlo 

plot(clust, hang = -1) # Con el argumento hang=-1, conseguimos que las etiquetas aparezcan alineadas por debajo de 0 en el eje

Si se decidiese cortar en el dendograma por una altura de seis en el eje de ordenadas, saldrían dos grupos, uno formado por Cataluña, Navarra, País Vasco, Aragón, La Rioja, Baleares y Madrid, y otro formado por el resto de las comunidades autónomas junto a Ceuta y Melilla.



In [ ]:
plot(clust, hang = -1)
abline(h=6, col="red")

Si el corte se hiciera en otro punto distinto, el número de grupos podría ser diferente, por ejemplo:



In [ ]:
par(mfrow=c(1,2))
plot(clust, hang = -1)
abline(h=4, col="red")
plot(clust, hang = -1)
abline(h=2.5, col="red")

¿Cómo saber cuál es la agrupación adecuada? Hay que elegir aquella que tenga sentido en el contexto que se está analizando, aunque podemos apoyarnos en algunas funciones que tiene R implementadas para este fin. La función NbClust del paquete NbClust sirve para este próposito, calculando el número de conglomerados a partir de varios índices:



In [ ]:
library(NbClust) # Determinar número de conglomerados
numero.conglomerado.complete <- NbClust(ccaa_tip, distance = "euclidean", min.nc = 2, max.nc = 11, method = "complete", index ="all")

#### Determinación del número óptimo de clusters

Utilizamos el paquete `NbClust` para decidir cuántos grupos son ideales.

In [ ]:
library(NbClust)
res_nc <- NbClust(ccaa_tip, distance = "euclidean", min.nc = 2, max.nc = 11, 
                  method = "complete", index = "all")

En la salida de esta función se muestra información sobre dos de los índices, Hubert y D-index. Para cada índice se ofrecen dos gráficos: el propio índice (en rojo), y las diferencias de los índices para números de conglomerados consecutivos (en azul). El número de conglomerados a seleccionar será aquel que presente una mayor diferencia (valor más alto en el gráfico azul). Según este criterio ambos índices nos indican que el número de conglomerados adecuado sería tres.

Además, esta salida también proporciona una tabla resumen con los resultados de los diferentes índices que calcula, así como una conclusión sobre el mejor número de conglomerados a utilizar.

Una vez decidido el número de conglomerados (3, en este caso), éstos se obtienen con la función cutree aplicada sobre el objeto clust obtenido con hclust.

In [ ]:
sol <- cutree(clust, k = 3)
# Mostrar asignación por comunidad
sort(sol)

Otra forma de presentar esta información es por conglomerado,



In [ ]:
sort(sol)


La solución de la función cutree junto con los datos originales ccaa los almacenaremos en un objeto al que llamamos resccaa para su posterior análisis.



In [ ]:
resccaa <- ccaa
resccaa$conglomerado <- sol

Si nos fijamos en la salida de la función cutree se puede ver como el primer grupo estaría formado por Andalucía, Asturias, Canarias, Cantabria, Castilla-León, Castilla-La Mancha, Comunidad Valenciana, Extremadura, Galicia y Murcia. El segundo conglomerado agruparía a Aragón, Islas Baleares, Cataluña, Madrid, Navarra, País Vasco y la Rioja, y el último grupo estaría formado por las ciudades autónomas de Ceuta y Melilla, como era de esperar al presentar estas dos ciudades valores atípicos en las tasas de emigración e inmigración. Obsérvese que el primer grupo es el más numeroso, con 10 de las 17 comunidades autónomas. Dentro de este grupo podrían apreciarse dos subgrupos, uno con comunidades del norte (Asturias, Cantabria, Castilla-León y Galicia), y otro con las comunidades del sur más las Islas Canarias (Andalucía, Canarias, Castilla-La Mancha, Comunidad Valenciana, Extremadura y Murcia).

Una manera de visualizar el resultado del análisis de conglomerados es con la función fviz_cluster del paquete factoextra,



In [ ]:
library(factoextra)
fviz_cluster(hcut(ccaa_tip, k = 3, hc_method = "complete"), palette = "rainbow_hcl")

También se podría utilizar un mapa de España para representar los conglomerados, ya que los individuos que se desean agrupar son las comunidades autónomas,



In [ ]:
library(colorspace) # color mapas
library(sf)

In [ ]:
library(sf)
library(colorspace)

k <- 3 # número de conglomerados

# 1. Asegurar que ccaasf es un objeto sf
ccaasf_sf <- st_as_sf(ccaasf)

# 2. Añadir la columna de unión a resccaa
resccaa$COM <- c("CA01","CA02","CA03","CA04","CA05","CA06","CA07","CA08","CA09",
                 "CA10","CA11","CA12","CA13","CA14","CA15","CA16","CA17","CA18","CA19")

# 3. Realizar el merge (sf mantiene la geometría automáticamente)
ccaasfC <- merge(ccaasf_sf, resccaa, by = "COM")

# 4. Definir colores
colores <- rainbow_hcl(k)

# 5. Graficar específicamente la columna del conglomerado
# En sf, si no especificas la columna, intenta graficar todas.
plot(ccaasfC["conglomerado"], 
     col = colores[as.factor(ccaasfC$conglomerado)], 
     main = "Mapa de Conglomerados CCAA",
     key.pos = 1) # Posición de la leyenda abajo

#### Conglomerado no Jerárquico con kmeans
Este método consiste en clasificar a los individuos en un número k
 de grupos fijado de antemano. Cada grupo viene representado por el vector de medias de las variables para los individuos que pertenecen a él, denominado centroide.

Una vez fijado k
, el algoritmo del método kmeans comienza asignando, de forma arbitaria, una observación a cada uno de los conglomerados. Esta asignación constituye el centroide inicial. Las observaciones restantes se van incorporando al grupo más cercano (en términos de distancia euclídea entre la observación y el centroide del grupo). Tras esta asignación, el algoritmo calcula de nuevo el centroide para cada conglomerado y reasigna las observaciones con el criterio de que estén incluidas en el conglomerado más cercano. Los centroides se van recalculando y el proceso se repite hasta que los conglomerados cambian poco entre etapas, o bien se llega al número máximo de iteraciones fijado a priori (número arbitrario).

Los principales problemas de este tipo de análisis son la determinación del número k
 de grupos y la selección de los k
 centros iniciales (ya que los resultados son sensibles a los puntos utilizados como inicio, a los denominados puntos semilla). Existen varias herramientas que nos ayudan a solventar en parte estas cuestiones:

Determinación del número k
 de grupos. Se puede utilizar el gráfico de sedimentación o un método de conglomerados jerárquico. El gráfico de sedimentación es la representación gráfica de la variabilidad dentro de los grupos para cada valor de k
, siendo k
 el número de grupos. Esta variabilidad desciende con el número de grupos, por lo que el k
 elegido será aquel en el que esta reducción se estabiliza (se observa un codo). El gráfico se obtiene con la función fviz_nbclust del paquete factoextra con la opción wss en el argumento method.


In [ ]:
fviz_nbclust(ccaa_tip, kmeans, method = "wss")  


El gráfico de sedimentación indica que el número de conglomerados a considerar podría ser 3 o 4, mientras que en el análisis jerárquico habíamos determinado la existencia de 3 grupos, uno con 2 subgrupos. Ambos resultados son compatibles.

Selección de los k
 centros iniciales. Se pueden considerar como centros de inicio los centroides de los grupos determinados en el análisis jerárquico, o repetir el algoritmo kmedias con distintos centros iniciales hasta encontrar una partición estable. La utilización de más de un conjunto de centros de inicio, reduce la dependencia de la partición de los puntos semilla.


La función kmeans permite considerar uno o varios conjuntos de centros de inicio fijados aleatoriamente, a través de su argumento nstart y establecer el número de grupos o quienes son los centroides iniciales con centers. Así, si se desea agrupar los individuos en k
 grupos (centers = k) con un único conjunto de centros de inicio aleatorios se utilizará nstart = 1 o con más de uno (nstart > 1), mientras que para utilizar los centroides obtenidos en un análisis jerárquico habrá que sustituir en el argumento centers el valor de k por el nombre del objeto que contiene a dichos centroides

##### a) kmeans con el número de grupos fijado a priori y con varios centros de inicio aleatorios.
El número de grupos considerados es tres y se fuerza al algoritmo a elegir 100 conjuntos de centros de inicio aleatorios. El algoritmo determina como solución aquella agrupación que minimiza la variabilidad dentro de los conglomerados.

In [ ]:
km1 <- kmeans(ccaa_tip, centers = 3, nstart = 100)
km1

Podemos observar que en la salida de esta función tenemos información, entre otras cosas, sobre el conglomerado al que pertenece cada comunidad autónoma, que se puede extraer directamente utilizando $cluster (ver Available components)



In [ ]:
km1$cluster  


Si además, la orden anterior se incluye dentro de la función sort aparecerán las comunidades autónomas ordenadas por conglomerado



In [ ]:
sort(km1$cluster) 


Los conglomerados identificados son los mismos que los obtenidos en el análisis jerárquico (observe que el grupo 1 del jeráquico es el grupo 3 del no jerárquico). Se puede realizar una visualización de los grupos sobre el espacio de las dos primeras componentes principales:



In [ ]:
fviz_cluster(km1, ccaa_tip, ellipse.type = "convex", palette = "rainbow_hcl")


#### kmeans utilizando los centroides del análisis jerárquico como centro de inicio (no aleatorio).
Para poder utilizar como punto de partida en el método de kmeans, el resultado obtenido en el análisis jerárquico con hclust, necesitamos calcular y guardar en un objeto los centroides de los 3 grupos obtenidos.

Esto se puede hacer de varias formas, con la función cls.attrib del paquete clv


In [ ]:
# Alternativa a clv para obtener los centros:
cent <- t(sapply(split(as.data.frame(ccaa_tip), sol), colMeans))
colnames(cent) <- colnames(ccaa_tip)
rownames(cent) <- paste("Cluster", 1:nrow(cent))

# Ahora ya puedes hacer el kmeans
km2 <- kmeans(ccaa_tip, centers = cent)

o programándolo nosotros mismos,



In [ ]:
# data frame con los datos originales tipificados y el grupo al que pertenece cada comunidad autónoma 
ccaa_tip.res <- as.data.frame(ccaa_tip)
ccaa_tip.res$conglomerado <- sol

# Cálculo de los centroides
centro <- NULL
centroides <- NULL
for (i in 1:3) {
  centro <- apply(subset(ccaa_tip.res, conglomerado ==i, select = c(1:7)), 2, mean)
  centroides <- rbind(centroides, centro) 
}
rownames(centroides) <- paste("centro", 1:3, sep = " ")
head(centroides)

Los objetos cent y centroides contienen la misma información, los centroides de los tres grupos determinados en el análisis jerárquico y que utilizaremos como centros de inicio en el análisis no jerárquico k-means.



In [ ]:
km2 <- kmeans(ccaa_tip, centers = cent)
sort(km2$cluster)

En este caso, coinciden tanto los conglomerados identificados como la denominación del grupo. La visualización de los grupos también se puede hacer gráficamente,



In [ ]:
fviz_cluster(km2, ccaa_tip, ellipse.type = "convex", palette = "rainbow_hcl")


Obsérvese que el objeto creado (km1 o km2) contiene información del tamaño y composición de los conglomerados ($size y $conglomerado), los centroides finales ($centers) y las sumas de cuadrados entre e intra conglomerados ($betweenss y $withinss).

